# Lab 4 - Model Training with AutoML

In this lab you will us the automated machine learning (Auto ML) capabilities within the Azure Machine Learning service to automatically train multiple models with varying algorithms and hyperparameters, select the best performing model and register that model.

## Download the datasets

The following cell will download the dataset used by this lab. Click into the following cell and use `Shift + Enter` to execute it

In [1]:
import os

main_path = os.path.abspath(os.path.curdir)
print("Current working directory is ", main_path)
data_path = os.path.join(main_path, 'data')
os.listdir(data_path)

Current working directory is  C:\Users\sasever\Desktop\SelfLearning\AzureML\AML-service-labs-master\starter-artifacts\jupyter\azure-ml-labs\04-automl


['UsedCars_Affordability.csv']

## Train a model using AutoML

This lab builds upon the lessons learned in the previous lab, but is self contained so you work thru this lab without having to run a previous lab.

In following cell you are loading the data prepared in previous labs and acquiring (or creating) an instance of your Azure Machine Learning Workspace. In this cell, be sure to set the values for `subscription_id`, `resource_group`, `workspace_name` and `workspace_region` as directed by the comments. Execute the cell.

In [2]:
# Step 1 - Load training data and prepare Workspace
###################################################
import os
import numpy as np
import pandas as pd
from sklearn import linear_model 
from sklearn.externals import joblib
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import azureml
from azureml.core import Run
from azureml.core import Workspace
from azureml.core.run import Run
from azureml.core.experiment import Experiment
from azureml.train.automl import AutoMLConfig
import pickle

# Verify AML SDK Installed
# view version history at https://pypi.org/project/azureml-sdk/#history 
print("SDK Version:", azureml.core.VERSION)


# Load our training data set
pathToCsvFile = os.path.join(data_path, 'UsedCars_Affordability.csv')
df_affordability = pd.read_csv(pathToCsvFile, delimiter=',')
print(df_affordability)

full_X = df_affordability[["Age", "KM"]]
full_Y = df_affordability[["Affordable"]]

# Create a Workspace

# Create a new Workspace or retrieve the existing one
#Provide the Subscription ID of your existing Azure subscription
subscription_id ='757c4165-0823-49f7-9678-5a85fe5e13cc'

# Provide values for the Resource Group and Workspace that will be created
resource_group = 'MLworkspace2'
workspace_name = 'snml2'
workspace_region = 'westeurope'  # eastus, westcentralus, southeastasia, australiaeast, westeurope


# By using the exist_ok param, if the worskpace already exists we get a reference to the existing workspace instead of an error
ws = Workspace.create(
    name = workspace_name,
    subscription_id = subscription_id,
    resource_group = resource_group, 
    location = workspace_region,
    exist_ok = True)

print("Workspace Provisioning complete.")


SDK Version: 1.0.6
      Age     KM  Affordable
0      23  46986           0
1      23  72937           0
2      24  41711           0
3      26  48000           0
4      30  38500           0
5      32  61000           0
6      27  94612           0
7      30  75889           0
8      27  19700           0
9      23  71138           0
10     25  31461           0
11     22  43610           0
12     25  32189           0
13     31  23000           0
14     32  34131           0
15     28  18739           0
16     30  34000           0
17     24  21716           0
18     24  25563           0
19     30  64359           0
20     30  67660           0
21     29  43905           0
22     28  56349           0
23     28  32220           0
24     29  25813           0
25     25  28450           0
26     27  34545           0
27     29  41415           0
28     28  44142           0
29     30  11090           0
...   ...    ...         ...
1406   70  44850           1
1407   69  44826        

To train a model using AutoML you need only provide a configuration for AutoML that defines items such as the type of model (classification or regression), the performance metric to optimize, exit criteria in terms of max training time and iterations and desired performance, any algorithms that should not be used, and the path into which to output the results. This configuration is specified using the `AutomMLConfig` class, which is then used to drive the submission of an experiment via `experiment.submit`.  When AutoML finishes the parent run, you can easily get the best performing run and model from the returned run object by using `run.get_output()`. Execute the following cell to define the helper function that wraps the AutoML job submission.

In [11]:
# Step 2 - Define a helper method that will use AutoML to train multiple models and pick the best one
##################################################################################################### 
def auto_train_model(ws, experiment_name, model_name, full_X, full_Y,training_set_percentage, training_target_accuracy):

    # start a training run by defining an experiment
    experiment = Experiment(ws, experiment_name)
    
    train_X, test_X, train_Y, test_Y = train_test_split(full_X, full_Y, train_size=training_set_percentage, random_state=42)

    train_Y_array = train_Y.values.flatten()

    # Configure the automated ML job
    # The model training is configured to run on the local machine
    # The values for all settings are documented at https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-configure-auto-train
    # Notice we no longer have to scale the input values, as Auto ML will try various data scaling approaches automatically
    Automl_config = AutoMLConfig(task = 'classification',
                                 primary_metric = 'accuracy',
                                 max_time_sec = 12000,
                                 iterations = 20,
                                 n_cross_validations = 3,
                                 exit_score = training_target_accuracy,
                                 blacklist_algos = ['kNN','LinearSVM'],
                                 X = train_X,
                                 y = train_Y_array,
                                 path='.\\outputs')

    # Execute the job
    run = experiment.submit(Automl_config, show_output=True)

    # Get the run with the highest accuracy value.
    best_run, best_model = run.get_output()

    return (best_model, run, best_run)

In the following cell, you invoke the AutoML job to begin the training. Execute the following cell.

In [13]:
# Step 3 - Execute the AutoML driven training
#############################################
experiment_name = "Experiment-AutoML-04"
model_name = "usedcarsmodel"
training_set_percentage = 0.50
training_target_accuracy = 0.93
best_model, run, best_run = auto_train_model(ws, experiment_name, model_name, full_X, full_Y, training_set_percentage, training_target_accuracy)

# Examine some of the metrics for the best performing run
import pprint
pprint.pprint({k: v for k, v in best_run.get_metrics().items() if isinstance(v, float)})

/databricks/python/lib/python3.5/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
 FutureWarning)
WARNING:root:Parameter 'max_time_sec' will be deprecated. Use 'iteration_timeout_minutes'
WARNING:root:Parameter 'exit_score' will be deprecated. Use 'experiment_exit_score'
WARNING:root:Parameter 'blacklist_algos' will be deprecated. Use 'blacklist_models'
WARNING:root:Parameter 'max_time_sec' will be deprecated. Use 'iteration_timeout_minutes'
WARNING:root:Parameter 'exit_score' will be deprecated. Use 'experiment_exit_score'
WARNING:root:Parameter 'blacklist_algos' will be deprecated. Use 'blacklist_models'
Parent Run ID: AutoML_448f6dc8-19ad-49af-9d0f-c368f9f9f906
*******************************************************************************************
ITERATION: The iteration being evaluated.
PIPELINE: A summary description of the pipeline being evaluated.
DURATION: Time taken for the current iteration.
METRIC: The result of computing score on the fitted pipeline.
BEST: The best observed score thus far.
*******************************************************************************************

 ITERATION PIPELINE DURATION METRIC BEST
 0 StandardScalerWrapper KNN 0:00:06 0.9387 0.9387
Stopping criteria reached at iteration 1. Ending experiment.WARNING:root:Parameter 'blacklist_algos' will be deprecated. Use 'blacklist_models'
{'AUC_macro': 0.9684345902786315,
 'AUC_micro': 0.9684345902786315,
 'AUC_weighted': 0.9684345902786315,
 'accuracy': 0.9387494188749419,
 'accuracy_max': 0.9387494188749419,
 'average_precision_score_macro': 0.9868041720856932,
 'average_precision_score_micro': 0.9868041720856932,
 'average_precision_score_weighted': 0.9868041720856932,
 'balanced_accuracy': 0.8869954835400741,
 'f1_score_macro': 0.9024329049250029,
 'f1_score_micro': 0.9387494188749419,
 'f1_score_weighted': 0.9373955488052862,
 'log_loss': 0.3149414073434263,
 'norm_macro_recall': 0.7739909670801484,
 'precision_score_macro': 0.9209147723141714,
 'precision_score_micro': 0.9387494188749419,
 'precision_score_weighted': 0.9376006139940539,
 'recall_score_macro': 0.8869954835400741,
 'recall_score_micro': 0.9387494188749419,
 'recall_score_weighted': 0.9387494188749419,
 'weighted_accuracy': 0.9635993950946903}

Try out the best model by executing the following cell.

In [15]:
# Step 4 - Try the best model
#############################
age = 60
km = 40000
print(best_model.predict( [[age,km]] ))

[1]

## Register an AutoML created model

You can register models created by AutoML with Azure Machine Learning just as you would any other model. Execute the following cell to register this model.

In [18]:
# Step 5 - Register the best performing model for later use and deployment
#################################################################
# notice the use of the root run (not best_run) to register the best model
run.register_model(description='AutoML trained used cars classifier')

WARNING:root:Parameter 'blacklist_algos' will be deprecated. Use 'blacklist_models'
Registering model AutoML448f6dc81best
 Out[ 6 ]: <azureml.core.model.Model at 0x7f6c3c43a240>